In [ ]:
import numpy as np
import os
import argparse
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import cv2
import glob

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras import regularizers, optimizers

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D, SeparableConv2D, Add
from tensorflow.keras.layers import LeakyReLU, ReLU

In [ ]:
MODEL_DIR = "../assets/models/"
DATA_DIR = "../assets/data/"

In [ ]:
model_name = "cnn_113_80.h5f"
large = False
# model_name = "cnn_227_160.h5f"
# large = True

import re
p = re.compile(r"cnn_(\d+)_(\d+).h5f")
h, w = p.findall(model_name)[0]
h, w = int(h), int(w)

TARGET_SIZE = (h, w)

# ZOOM_FACTOR = 2
# TARGET_SIZE = (SRC_HEIGHT//ZOOM_FACTOR, SRC_WIDTH//ZOOM_FACTOR)

print(f"target_size = {TARGET_SIZE}")

OUTPUT_SHAPE = 3
BATCH_SIZE = 256

In [ ]:
RECORD_DIR = "../assets/data/records/"
EM_RECORD_DIR = "../assets/data/emulator_records/"

p = re.compile(r".*images-\d+-(\d+)\.tfrec")

X = glob.glob(os.path.join(RECORD_DIR, 'images-*.tfrec'))[:50]+\
    glob.glob(os.path.join(EM_RECORD_DIR, 'images-*.tfrec'))
X = filter(lambda x: len(p.findall(x)) > 0, X)
X = list(X)

N = sum(map(lambda x: int(p.findall(x)[0]), X))
print(N)

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE

def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    image = tf.image.resize(image, TARGET_SIZE)
    image = tf.reverse(image, axis=[-1])
    #image = tf.reshape(image, [*TARGET_SIZE, 3]) # explicit size needed for TPU
    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), 
        "image_name": tf.io.FixedLenFeature([], tf.string), 
        "x_center": tf.io.FixedLenFeature([], tf.float32),  
        "y_center": tf.io.FixedLenFeature([], tf.float32),  
        "confidence": tf.io.FixedLenFeature([], tf.float32),  
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = (example['x_center'], example['y_center'], example['confidence'])
    label = [tf.cast(x, tf.float32) for x in label]
    return image, label


def read_dataset(filenames):
    # automatically interleaves reads from multiple files
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) 

    opt = tf.data.Options()
    opt.experimental_deterministic = False
    dataset = dataset.with_options(opt) 

    dataset = dataset.map(read_labeled_tfrecord, num_parallel_calls=AUTO)
    
    return dataset

def data_augment(image, label):
    #image = tf.image.random_flip_left_right(image)
    #image = tf.image.random_saturation(image, 0, 2)
    return image, label 

def get_train_dataset(filenames):
    d = read_dataset(filenames)
#     d = d.map(data_augment, num_parallel_calls=AUTO)
    d = d.repeat()
    d = d.shuffle(2048)
    d = d.batch(BATCH_SIZE)
    d = d.prefetch(AUTO)
    return d

def get_test_dataset(filenames):
    d = read_dataset(filenames)
    d = d.batch(BATCH_SIZE)
    d = d.cache()
    d = d.prefetch(AUTO)
    return d

In [ ]:
full_dataset = read_dataset(X)
full_dataset = full_dataset.shuffle(10, reshuffle_each_iteration=False)

split_factor = 8

def is_test(x, y):
    return x % split_factor == 0

def is_train(x, y):
    return not is_test(x, y)

def recover(x, y):
    return y

test_dataset = full_dataset.enumerate() \
                    .filter(is_test) \
                    .map(recover)
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.cache()
test_dataset = test_dataset.prefetch(AUTO)

train_dataset = full_dataset.enumerate() \
                    .filter(is_train) \
                    .map(recover)
train_dataset = train_dataset.shuffle(2048)
train_dataset = train_dataset.batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(AUTO)
train_dataset = train_dataset.repeat()

In [ ]:
for images, labels in train_dataset.take(1):
    break
for i, (image, label) in enumerate(zip(images, labels)):
    if i == 2:
        print(label)
        plt.imshow(image)
        plt.show()
        break

In [ ]:
def detect_accuracy(y_true, y_pred, thresh=0.8):
    true_cls = y_true[:,2]
    pred_cls = y_pred[:,2]
    pred_cls = tf.math.greater(pred_cls, thresh)
    pred_cls = tf.cast(pred_cls, true_cls.dtype)
    
    abs_err = tf.math.abs(true_cls-pred_cls)
    return 1-tf.math.reduce_mean(abs_err)

def position_accuracy(y_true, y_pred):
    true_cls = y_true[:,2]
    true_pos = y_true[:,:2]
    pred_pos = y_pred[:,:2]
    
    abs_err = tf.math.abs(true_pos-pred_pos)
    dist_sqr_err = tf.math.reduce_sum(tf.math.square(abs_err), axis=1)
    dist_err = tf.math.sqrt(dist_sqr_err)
    
    # only consider when object is there
    dist_err = tf.math.multiply(dist_err, true_cls)
    net_err = tf.math.reduce_sum(dist_err)
    total_objects = tf.math.reduce_sum(true_cls)
    
    mean_err = net_err / total_objects
    return 1-mean_err

In [ ]:
def create_model(large=False):
    alpha = 0.2
    dropout = 0.1

    inputs = Input(shape=TARGET_SIZE+(3,))

#    x_skip = inputs
    
    x = Conv2D(16, (3, 3))(inputs)
    x = ReLU()(x)
    x = MaxPooling2D((2, 2))(x)

    x = SeparableConv2D(32, (3, 3))(x)
    x = ReLU()(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(32, (3, 3))(x)
    x = ReLU()(x)
    x = MaxPooling2D((2, 2))(x)

    x = SeparableConv2D(64, (3, 3))(x)
    x = ReLU()(x)
    
    if large:
        x = MaxPooling2D((2, 2))(x)
    else:
        x = MaxPooling2D((3, 3))(x)
    
#     x_skip = Conv2D(64, (16, 16))(x_skip)
#     x_skip = MaxPooling2D((17,17))(x_skip)
#     x = Add()([x, x_skip])

    # larger network
    if large:
        x = Conv2D(64, (3, 3))(x)
        x = ReLU()(x)
        x = MaxPooling2D((2, 2))(x)

        x = SeparableConv2D(128, (3, 3), padding="same")(x)
        x = ReLU()(x)
        x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)

    if large:
        x = Dense(128)(x)
        x = ReLU()(x)
    
    x = Dense(64)(x)
    x = ReLU()(x)

    x = Dense(32)(x)
    x = ReLU()(x)

    x = Dense(OUTPUT_SHAPE)(x)
    outputs = x
    #outputs = LeakyReLU(alpha)(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

model = create_model(large=large)

model.load_weights(os.path.join(MODEL_DIR, model_name))
#model.load_weights(os.path.join(MODEL_DIR, "cnn_227_160.h5f"))

model.compile(
    optimizer=keras.optimizers.Adam(lr=1e-3),
    loss="mae",
    metrics=[detect_accuracy, position_accuracy],
)

print(model.summary())

In [ ]:
n_train = int((split_factor-1)/split_factor*N)
n_test = int(1/split_factor*N)

steps_per_epoch = n_train // BATCH_SIZE
validation_steps = n_test // BATCH_SIZE

print("steps per epoch: {0}".format(steps_per_epoch))
print("validation steps {0}".format(validation_steps))

TOTAL_EPOCHS = 5

model.fit(
    train_dataset, 
    steps_per_epoch=steps_per_epoch,
    epochs=TOTAL_EPOCHS,
    validation_data=test_dataset)

In [ ]:
model.save_weights(os.path.join(MODEL_DIR, model_name))
# model.save_weights("../models/cnn_227_160.h5f")
# model.save_weights("../models/cnn_113_80.h5f")

In [ ]:
y_pred = model.predict(images)
y_true = labels
print(detect_accuracy(y_pred, y_true))
print(position_accuracy(y_pred, y_true))